In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
import os
import torch

env = gym.make("LunarLander-v3", render_mode="human")

observation, info = env.reset(seed=42)
for _ in range(5):
    
    action = env.action_space.sample()

    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()

env.close()

In [29]:
log_path=os.path.join('Training','Logs')

In [30]:
env = gym.make("LunarLander-v3", render_mode="human")
devive=torch.device('cuda' if torch.cuda.is_available else'cpu')
model=PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path).to(device)
model.learn(total_timesteps=100_000)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to Training\Logs\PPO_1


d:\miniconda flie\envs\RL\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 90.1     |
|    ep_rew_mean     | -234     |
| time/              |          |
|    fps             | 46       |
|    iterations      | 1        |
|    time_elapsed    | 43       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 90.5        |
|    ep_rew_mean          | -226        |
| time/                   |             |
|    fps                  | 46          |
|    iterations           | 2           |
|    time_elapsed         | 87          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.006834901 |
|    clip_fraction        | 0.0324      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38       |
|    explained_variance   | 0.00367     |
|    learning_rate        | 0.

In [31]:
PPO_Path=os.path.join('Training','Saved Models','PPO_Model_LunarLander')

In [32]:
model.save(PPO_Path)

In [33]:
del model

In [34]:
model=PPO.load(PPO_Path,env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


d:\miniconda flie\envs\RL\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [35]:
evaluate_policy(model,env,n_eval_episodes=10,render=True)

d:\miniconda flie\envs\RL\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(56.670992330373814, 63.93916236773709)

In [ ]:
episodes=5
for episode in range(episodes+1):
    obs, info = env.reset()
    done=False
    score=0

    while not done:
        gym.make('LunarLander-v3',render_mode='human')
        env.render()
        action, _ = model.predict(obs)
        observation, reward, terminated, truncated, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode,score))
env.close()

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback,StopTrainingOnRewardThreshold

In [ ]:
save_path=os.path.join('Training','Saved Model')

In [ ]:
stop_callback=StopTrainingOnRewardThreshold(reward_threshold=200,verbose=1)
eval_callback=EvalCallback(env,
                           callback_on_new_best=stop_callback,
                           eval_freq=10000,
                           best_model_save_path=save_path,
                           verbose=1)

In [ ]:
model=PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path)